# **This creates the lower scored of my submissions - Topics look better, but accuracy suffers**

In [1]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords, wordnet
import nltk;
from nltk.stem import WordNetLemmatizer
from gensim.models import ldamodel
from gensim.models.hdpmodel import HdpModel
from gensim.models import CoherenceModel
from gensim import matutils, models
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import scipy.sparse
import string
import pickle;
import re;

In [2]:
stp = [i.lower() for i in stopwords.words('english')]
lemmatizer = WordNetLemmatizer()

mainData = pd.read_csv("../input/unstructured-l0-nlp-hackathon/data.csv")

In [3]:
def get_wordnet_pos(treebank_tag):
    '''
    Function that takes in nltk POS tags and returns tags so that they can be used for
    lemmatizaition
    '''
#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
    if treebank_tag.startswith('N'):
        return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
    else:
        return None

In [4]:
def get_lda_topics(model, num_topics):
    word_dict = {}
    topics = model.show_topics(num_topics,20)
    word_dict = {'Topic '+str(i):[x.split('*') for x in words.split('+')] \
                 for i,words in model.show_topics(10,20)}
    return pd.DataFrame.from_dict(word_dict)

In [5]:
def get_hdp_topics(model, num_topics):
    word_dict = {}
    topics = model.show_topics(num_topics,20)
    word_dict = {'Topic '+str(i):[x.split('*') for x in words.split('+')] \
                 for i,words in model.show_topics(10,20)}
    return pd.DataFrame.from_dict(word_dict)

In [6]:
def cleaner_array(A):
    tokens = nltk.word_tokenize(A)
    postags = nltk.pos_tag(tokens)

    resp_list = []

    for elem,tag in postags:
        if elem.lower().strip() not in stp and len(elem)>2:
            try:
                if get_wordnet_pos(tag) != None:
                    resp_list.append(lemmatizer.lemmatize(elem.lower().strip(), get_wordnet_pos(tag)))
            except:
                print(elem, tag)

    return(" ".join(resp_list))

In [7]:
mainData["clean"] = mainData["text"].apply(lambda x: cleaner_array(x))
mainData["clean"] = mainData["clean"].str.lower().apply(lambda x: re.sub(r'(@[\S]+)|(\w+:\/\/\S+)|(\d+)','',x))

In [8]:
mainData.columns

Index(['Id', 'text', 'clean'], dtype='object')

In [9]:
corpus = mainData["clean"].tolist()

# Using TF-IDF values to find my BOW and running LDA

tf_vectorizer = TfidfVectorizer()
tf_idf_model = tf_vectorizer.fit_transform(corpus)

DF_valid = pd.DataFrame(tf_idf_model.toarray(), columns=tf_vectorizer.get_feature_names())
DF_valid.index = mainData.Id

corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(DF_valid.transpose()))

Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in tf_vectorizer.vocabulary_.items())

lda = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=50)

get_lda_topics(lda,20)

Topics arent as clear as would be liked - Let us try Counts instead (for similarity to how LDAmodel works)

# Using Counts instead

In [10]:
countvect = CountVectorizer(stop_words='english', max_features = 4000, max_df = 0.8)
countvect_model = countvect.fit_transform(corpus)

In [11]:
data_count = pd.DataFrame(countvect_model.toarray(), columns=countvect.get_feature_names())
data_count.index = mainData.Id

In [12]:
corpus_count = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_count.transpose()))


id2word_count = dict((v, k) for k, v in countvect.vocabulary_.items())

In [13]:
lda_mod = models.LdaModel(corpus=corpus_count,
                          num_topics=5,
                          id2word=id2word_count,
                          random_state=1,
                          passes=50,#50,
                          alpha=0.001,
                          eta=1)

In [14]:
get_lda_topics(lda_mod,20)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,"[0.011, ""season"" ]","[0.036, ""car"" ]","[0.008, ""company"" ]","[0.071, ""pro"" ]","[0.020, ""place"" ]"
1,"[ 0.010, ""league"" ]","[ 0.018, ""line"" ]","[ 0.008, ""today"" ]","[ 0.033, ""company"" ]","[ 0.020, ""home"" ]"
2,"[ 0.009, ""team"" ]","[ 0.017, ""organization"" ]","[ 0.007, ""designer"" ]","[ 0.027, ""work"" ]","[ 0.017, ""room"" ]"
3,"[ 0.007, ""city"" ]","[ 0.013, ""article"" ]","[ 0.006, ""game"" ]","[ 0.026, ""people"" ]","[ 0.015, ""bedroom"" ]"
4,"[ 0.007, ""player"" ]","[ 0.008, ""university"" ]","[ 0.005, ""year"" ]","[ 0.025, ""employee"" ]","[ 0.015, ""minute"" ]"
5,"[ 0.006, ""game"" ]","[ 0.008, ""cur"" ]","[ 0.005, ""request"" ]","[ 0.023, ""benefit"" ]","[ 0.012, ""house"" ]"
6,"[ 0.006, ""year"" ]","[ 0.007, ""time"" ]","[ 0.005, ""time"" ]","[ 0.020, ""management"" ]","[ 0.011, ""bed"" ]"
7,"[ 0.006, ""club"" ]","[ 0.006, ""engine"" ]","[ 0.004, ""service"" ]","[ 0.014, ""honeywell"" ]","[ 0.010, ""mile"" ]"
8,"[ 0.006, ""week"" ]","[ 0.006, ""distribution"" ]","[ 0.004, ""google"" ]","[ 0.014, ""year"" ]","[ 0.010, ""restaurant"" ]"
9,"[ 0.006, ""time"" ]","[ 0.006, ""year"" ]","[ 0.004, ""day"" ]","[ 0.014, ""job"" ]","[ 0.010, ""area"" ]"


Let us validate the results:
* 2: "glassdoor_reviews",
* 3: "Automobiles",
* 4: "sports_news",
* 1: "tech_news",
* 0: "room_rentals"

Seems moderately appropriate - let us consider lda_mod as final model

In [15]:
corpus_transformed = lda_mod[corpus_count]

topics = []

for i in range(len(corpus_transformed)):
    v=dict(corpus_transformed[i])
    for top, score in v.items():
        if score == max(v.values()):
            topics.append(top)

In [16]:
final_output = pd.DataFrame(topics, index = mainData.Id, columns = ["topic"])

final_output.reset_index(inplace = True)

In [17]:
final_output.replace({'topic': {2: "glassdoor_reviews",
                                3: "Automobiles",
                                4: "sports_news",
                                1: "tech_news",
                                0: "room_rentals"}},
                     inplace = True)

In [18]:
final_output.to_csv("../input/output/submission_V9.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../input/output/submission_V9.csv'